<a href="https://colab.research.google.com/github/ConanGoodwin/AIRBNB_Chat_bot/blob/main/AIRBNB_Chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Primeiro bloco, de instalação de bibliotecas não default utilizadas

In [53]:
!pip install -q -U google-generativeai

Bloco de importação das bibliotecas utilizadas e definições das variaveis para utilização da Gemini

In [99]:
import google.generativeai as genai
import textwrap
from IPython.display import display, Markdown
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('SECRET_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

generation_config = {
    "candidate_count": 1,
    "temperature": 0.4,
}

Definições de funções auxiliares

In [100]:
def to_markdown(text):
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def set_filter(text):
  if text == '1':
    return 'BLOCK_NONE'
  elif text == '2':
    return 'BLOCK_ONLY_HIGH'
  elif text == '3':
    return 'BLOCK_MEDIUM_AND_ABOVE'
  elif text == '4':
    return 'BLOCK_LOW_AND_ABOVE'
  else:
    print("\n Nivel invalido, o nivel deve ser um numero entre 1 e 4!")
    return None

def safety_settings_level():
  print('\nVamos começar configurando os filtros de segurança.')
  SEXUAL = input('Defina o nivel de filtro de conteudo sexual de 1 a 4: ')
  SEXUAL = set_filter(SEXUAL)

  while SEXUAL is None:
    SEXUAL = input('Defina o nivel de filtro de conteudo sexual de 1 a 4: ')
    SEXUAL = set_filter(SEXUAL)

  display(to_markdown(f'Nivel escolhido: **{SEXUAL}**'))

  DANGER = input('Defina o nivel de filtro de conteudo com risco de 1 a 4: ')
  DANGER = set_filter(DANGER)

  while DANGER is None:
    DANGER = input('Defina o nivel de filtro de conteudo com risco de 1 a 4: ')
    DANGER = set_filter(DANGER)

  display(to_markdown(f'Nivel escolhido: **{DANGER}**'))

Tela inicial com opção ao usuario de definição dos niveis de segurança para conteudo sexual e de risco.

In [101]:
display(to_markdown(f'# Bem Vindo ao seu turismo AirBnb personalizado!'))
display(to_markdown(f'### através de apenas algumas perguntas, disponibilizamos para você um roteiro personalizado baseado no seu local de estadia!'))
display(to_markdown(f'---'))
print("Pressione ENTER para continuar...")
input()

SEXUAL = 'BLOCK_LOW_AND_ABOVE'
DANGER = 'BLOCK_LOW_AND_ABOVE'

print(f'deseja configurar o nivel de segurança do conteudo? \033[1mpor padrão todo conteudo de risco é bloqueado.\033[0m')
display(to_markdown(f'Digite **S** ou **N**  '))
is_safe = input('>>>')

if is_safe.lower() == 's':
  safety_settings_level()

safety_settings = {
    "HARASSMENT": "BLOCK_LOW_AND_ABOVE",
    "HATE": "BLOCK_LOW_AND_ABOVE",
    "SEXUAL": SEXUAL,
    "DANGEROUS": DANGER,
}

> # Bem Vindo ao seu turismo AirBnb personalizado!

> ### através de apenas algumas perguntas, disponibilizamos para você um roteiro personalizado baseado no seu local de estadia!

> ---

Pressione ENTER para continuar...

deseja configurar o nivel de segurança do conteudo? por padrão todo conteudo de risco é bloqueado.


> Digite **S** ou **N**  

>>>s

Vamos começar configurando os filtros de segurança.
Defina o nivel de filtro de conteudo sexual de 1 a 4: 3


> Nivel escolhido: **BLOCK_MEDIUM_AND_ABOVE**

Defina o nivel de filtro de conteudo com risco de 1 a 4: 3


> Nivel escolhido: **BLOCK_MEDIUM_AND_ABOVE**

inicialização do modelo com as configurações setadas

In [102]:
model = genai.GenerativeModel(model_name="gemini-1.0-pro",safety_settings=safety_settings,generation_config=generation_config)

Definições dos dados da localidade da hospedagem, com verificação de localidades validas

In [104]:
display(to_markdown(f'### 1.configurações de localidade'))
display(to_markdown(f'---'))

estado=input(f'Digite o nome do ESTADO onde se localiza a hospedagem: ')
check = 's'
response = model.generate_content(f'responda somente com sim ou não, sem explicação, se {estado} é um estado valido do Brasil')
if response.text.lower() == 'não':
  check = input(f'{estado} não parece ser um estado valido do Brasil, deseja continuar assim mesmo?(s/n)\n ')
# print(response.text)

while check.lower() != 's':
  # print(response.text)
  estado=input(f'Digite o nome do ESTADO onde se localiza a hospedagem: ')
  response = model.generate_content(f'responda somente com sim ou não, sem explicação, se {estado} é um estado valido do Brasil')
  if response.text.lower() == 'não':
    check = input(f'{estado} não parece ser um estado valido do Brasil, deseja continuar assim mesmo?(s/n)\n ')
  else:
    check = 's'

cidade=input(f'Digite o nome da CIDADE onde se localiza a hospedagem: ')
response = model.generate_content(f'responda somente com sim ou não, sem explicação, se {cidade} é uma cidade valida do {estado} no Brasil')
if response.text.lower() == 'não':
  check = input(f'{cidade} não parece ser uma cidade valida no estado {estado} do Brasil, deseja continuar assim mesmo?(s/n)\n ')
# print(response.text)

while check.lower() != 's':
  # print(response.text)
  cidade=input(f'Digite o nome da CIDADE onde se localiza a hospedagem: ')
  response = model.generate_content(f'responda somente com sim ou não, sem explicação, se {cidade} é uma cidade valida do {estado} no Brasil')
  if response.text.lower() == 'não':
    check = input(f'{cidade} não parece ser uma cidade valida no estado {estado} do Brasil, deseja continuar assim mesmo?(s/n)\n ')
  else:
    check = 's'

bairro=input(f'Digite o nome do BAIRRO onde se localiza a hospedagem: ')
response = model.generate_content(f'responda somente com sim ou não, sem explicação, se {bairro} é um bairro valido na {cidade} do {estado} no Brasil')
if response.text.lower() == 'não':
  check = input(f'{bairro} não parece ser um bairro valido na {cidade}, deseja continuar assim mesmo?(s/n)\n ')
# print(response.text)

while check.lower() != 's':
  # print(response.text)
  bairro=input(f'Digite o nome do ESTADO onde se localiza a hospedagem: ')
  response = model.generate_content(f'responda somente com sim ou não, sem explicação, se {bairro} é um bairro valido na {cidade} do {estado} no Brasil')
  if response.text.lower() == 'não':
    check = input(f'{bairro} não parece ser um bairro valido na cidade {cidade}, deseja continuar assim mesmo?(s/n)\n ')
  else:
    check = 's'

> ### 1.configurações de localidade

> ---

Digite o nome do ESTADO onde se localiza a hospedagem: rio de janeiro
rio de janeiro não parece ser um estado valido do Brasil, deseja continuar assim mesmo?(s/n)
 s
Digite o nome da CIDADE onde se localiza a hospedagem: rio de janeiro
Digite o nome do BAIRRO onde se localiza a hospedagem: paqueta


Definições das preferencias de lazer do usuario

In [105]:
display(to_markdown(f'### 2.configurações de preferencias de turismo'))
display(to_markdown(f'---'))

lista_lazer = []
type_lazer = ''

while type_lazer != 'sem':
  type_lazer=input(f'Digite uma nova categoria de lazer de interesse ou "fim" para encerrar\n(exs.: Cultura, Esportes, Aventura, Gastronômico, sol e praia, Ecológico, Religioso): ')
  check = 's'
  response = model.generate_content(f'exemplos de atividades de turismo validas: Cultura, Esportes, Aventura, Gastronômico, sol e praia, Ecológico, Religioso. responda somente com sim ou não, sem explicação, se {type_lazer} é uma atividade de turismo valida')

  if response.parts == []:
    display(to_markdown('**OPÇÃO INVALIDA DE ACORDO COM OS FILTROS DE SEGURAÇA**'))
  else:
    if response.text.lower() == 'não':
      check = input(f'{type_lazer} não parece ser uma atividade de turismo valida, deseja continuar assim mesmo?(s/n)\n ')

  while check.lower() != 's':
    type_lazer=input(f'Digite uma outra categoria de lazer de interesse\n(exs.: Cultura, Esportes, Aventura, Gastronômico, sol e praia, Ecológico, Religioso): ')
    response = model.generate_content(f'responda somente com sim ou não, sem explicação, se {type_lazer} é uma atividade de turismo valida')
    if response.parts == []:
      display(to_markdown('**OPÇÃO INVALIDA DE ACORDO COM OS FILTROS DE SEGURAÇA**'))
      check = 's'
    else:
      if response.text.lower() == 'não':
        check = input(f'{type_lazer} não parece ser uma atividade de turismo valida, deseja continuar assim mesmo?(s/n)\n ')
      else:
        check = 's'

  if type_lazer.lower() == 'fim':
    type_lazer = 'sem'
  elif response.parts != []:
    lista_lazer.append(type_lazer)

print(lista_lazer)

> ### 2.configurações de preferencias de turismo

> ---

Digite uma nova categoria de lazer de interesse ou "fim" para encerrar
(exs.: Cultura, Esportes, Aventura, Gastronômico, sol e praia, Ecológico, Religioso): Cultura
Digite uma nova categoria de lazer de interesse ou "fim" para encerrar
(exs.: Cultura, Esportes, Aventura, Gastronômico, sol e praia, Ecológico, Religioso): sol e praia
Digite uma nova categoria de lazer de interesse ou "fim" para encerrar
(exs.: Cultura, Esportes, Aventura, Gastronômico, sol e praia, Ecológico, Religioso): gastronomico
Digite uma nova categoria de lazer de interesse ou "fim" para encerrar
(exs.: Cultura, Esportes, Aventura, Gastronômico, sol e praia, Ecológico, Religioso): fim
['Cultura', 'sol e praia', 'gastronomico']


Exibição de dicas/roteiros de acordo com as escolhas dos usuarios

In [106]:
display(to_markdown(f'### 3.Roteiro, dicas e sugestões'))
display(to_markdown(f'---'))
type_roteiro = ''

chat = model.start_chat(history=[])

while type_roteiro != '3':
  display(to_markdown("""

  #### <ins>Lista de opções de tipos para a sugestões de turismo</ins>
  1. Dicas e sugestões livres.
  2. Roteiro baseado na quantidade de dias hospedados.
  3. Finalizar aplicação

  OBS.: As sugestões geradas se baseam em analise feita inteiramente por IA, confirme veracidade das sugestões com o host de sua hospedagem ou pessoas locais capacitadas para tal.
  """))
  type_roteiro = input(f'\n Digite o numero da opção para qual tipo de formato de sugestão turistica você deseja: ')

  while type_roteiro != '1' and type_roteiro !='2' and type_roteiro !='3':
    display(to_markdown('#### Opção inválida, digite 1 ou 2.'))
    type_roteiro = input(f'Digite o numero da opção para qual tipo de formato de sugestão turistica você deseja: ')

  if type_roteiro == '3':
    display(to_markdown('---'))
    display(to_markdown('---'))
    display(to_markdown('## FIM DA APLICAÇÃO'))
  else:

    if type_roteiro == '1':
      response = chat.send_message(f'de sugestões, indepedente de dias a serem realizadas, de atividades turisticas, no bairro de {bairro}, na cidade {cidade}, no estado {estado} no Brasil, baseado na seguinte lista de preferencias de atividades turisticas: {lista_lazer}')
    elif type_roteiro == '2':
      qtDias = input('Digite a quantidade de dias que pretende ficar hospedado: ')
      response = chat.send_message(f'faça um roteiro turistico, como se fosse um guia turistico, separado em {qtDias} dias, no bairro {bairro}, na cidade {cidade}, no estado {estado} no Brasil, baseado na seguinte lista de preferencias de atividades turisticas: {lista_lazer}')

    prompt = ''
    while prompt.lower() != "fim":
      print('____________________________________________________________________')
      display(to_markdown(f'**Resposta** >>> \n {response.text} \n'))
      print(f'Didite "fim" caso queira encerrar esta opção ou use o prompt para alguma duvida.')
      prompt = input('Digite um prompt: ')
      if prompt != 'fim':
        response = chat.send_message(prompt)

    print(f'\nSugestões finalizadas!')

> ### 3.Roteiro, dicas e sugestões

> ---

> 
> 
>   #### <ins>Lista de opções de tipos para a sugestões de turismo</ins>
>   1. Dicas e sugestões livres.
>   2. Roteiro baseado na quantidade de dias hospedados.
>   3. Finalizar aplicação
> 
>   OBS.: As sugestões geradas se baseam em analise feita inteiramente por IA, confirme veracidade das sugestões com o host de sua hospedagem ou pessoas locais capacitadas para tal.
>   


 Digite o numero da opção para qual tipo de formato de sugestão turistica você deseja: 1
____________________________________________________________________


> **Resposta** >>> 
>  **Cultura:**
> 
> * **Visitar o Museu Histórico de Paquetá:** Aprenda sobre a história e a cultura da ilha, incluindo sua importância como destino de veraneio no século XIX.
> * **Explorar a Igreja de São Roque:** Admire a arquitetura colonial desta igreja histórica, construída em 1740.
> * **Assistir a um espetáculo no Teatro Municipal de Paquetá:** Desfrute de apresentações de música, dança e teatro neste charmoso teatro.
> 
> **Sol e Praia:**
> 
> * **Relaxar na Praia da Moreninha:** Desfrute das águas calmas e da areia branca desta praia popular.
> * **Passear pela Praia da Guarda:** Explore esta praia tranquila e aproveite as vistas da Baía de Guanabara.
> * **Fazer um passeio de barco pela Baía de Guanabara:** Admire a beleza da ilha e da baía a partir da água.
> 
> **Gastronômico:**
> 
> * **Experimentar a culinária local no Restaurante Paquetá:** Saboreie pratos tradicionais como peixe assado, moqueca e feijoada.
> * **Visitar a Feira de Artesanato e Gastronomia:** Explore barracas que oferecem produtos artesanais e iguarias locais.
> * **Fazer um passeio gastronômico:** Conheça os melhores restaurantes e bares da ilha, experimentando diferentes sabores e especialidades. 


...

> 
> 
>   #### <ins>Lista de opções de tipos para a sugestões de turismo</ins>
>   1. Dicas e sugestões livres.
>   2. Roteiro baseado na quantidade de dias hospedados.
>   3. Finalizar aplicação
> 
>   OBS.: As sugestões geradas se baseam em analise feita inteiramente por IA, confirme veracidade das sugestões com o host de sua hospedagem ou pessoas locais capacitadas para tal.
>   


 Digite o numero da opção para qual tipo de formato de sugestão turistica você deseja: 3


> ---

> ---

> ## FIM DA APLICAÇÃO

Exibe o historico de solicitações com a finalização da apçicação

In [107]:
for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
  print('------------------------------------------------------------')

> **user**: de sugestões, indepedente de dias a serem realizadas, de atividades turisticas, no bairro de paqueta, na cidade rio de janeiro, no estado rio de janeiro no Brasil, baseado na seguinte lista de preferencias de atividades turisticas: ['Cultura', 'sol e praia', 'gastronomico']

------------------------------------------------------------


> **model**: **Cultura:**
> 
> * **Visitar o Museu Histórico de Paquetá:** Aprenda sobre a história e a cultura da ilha, incluindo sua importância como destino de veraneio no século XIX.
> * **Explorar a Igreja de São Roque:** Admire a arquitetura colonial desta igreja histórica, construída em 1740.
> * **Assistir a um espetáculo no Teatro Municipal de Paquetá:** Desfrute de apresentações de música, dança e teatro neste charmoso teatro.
> 
> **Sol e Praia:**
> 
> * **Relaxar na Praia da Moreninha:** Desfrute das águas calmas e da areia branca desta praia popular.
> * **Passear pela Praia da Guarda:** Explore esta praia tranquila e aproveite as vistas da Baía de Guanabara.
> * **Fazer um passeio de barco pela Baía de Guanabara:** Admire a beleza da ilha e da baía a partir da água.
> 
> **Gastronômico:**
> 
> * **Experimentar a culinária local no Restaurante Paquetá:** Saboreie pratos tradicionais como peixe assado, moqueca e feijoada.
> * **Visitar a Feira de Artesanato e Gastronomia:** Explore barracas que oferecem produtos artesanais e iguarias locais.
> * **Fazer um passeio gastronômico:** Conheça os melhores restaurantes e bares da ilha, experimentando diferentes sabores e especialidades.

------------------------------------------------------------


> **user**: como é a moreninha?

...

------------------------------------------------------------


> **user**: faça um roteiro turistico, como se fosse um guia turistico, separado em 5 dias, no bairro paqueta, na cidade rio de janeiro, no estado rio de janeiro no Brasil, baseado na seguinte lista de preferencias de atividades turisticas: ['Cultura', 'sol e praia', 'gastronomico']

------------------------------------------------------------


> **model**: **Roteiro Turístico de 5 Dias em Paquetá**
> 
> **Dia 1**
> 
> * **Manhã:** Visite o Museu Histórico de Paquetá para aprender sobre a história e a cultura da ilha.
> * **Tarde:** Relaxe na Praia da Moreninha, aproveitando as águas calmas e a areia branca.
> * **Noite:** Jante no Restaurante Paquetá para experimentar a culinária local, como peixe assado e moqueca.
> 
> **Dia 2**
> 
> * **Manhã:** Explore a Igreja de São Roque, admirando sua arquitetura colonial.
> * **Tarde:** Faça um passeio de barco pela Baía de Guanabara para apreciar a beleza da ilha e da baía.
> * **Noite:** Assista a um espetáculo no Teatro Municipal de Paquetá, desfrutando de apresentações de música, dança ou teatro.
> 
> **Dia 3**
> 
> * **Manhã:** Visite a Feira de Artesanato e Gastronomia para explorar barracas que oferecem produtos artesanais e iguarias locais.
> * **Tarde:** Faça um passeio gastronômico, conhecendo os melhores restaurantes e bares da ilha e experimentando diferentes sabores e especialidades.
> * **Noite:** Relaxe em um dos bares da orla, apreciando a vista da Baía de Guanabara.
> 
> **Dia 4**
> 
> * **Manhã:** Caminhe até a Praia da Guarda, uma praia tranquila com vistas deslumbrantes da Baía de Guanabara.
> * **Tarde:** Alugue um caiaque ou stand-up paddle e explore a baía a partir da água.
> * **Noite:** Experimente a culinária internacional em um dos restaurantes da ilha, oferecendo opções como comida italiana, japonesa e tailandesa.
> 
> **Dia 5**
> 
> * **Manhã:** Faça uma visita guiada à Ilha de Paquetá, aprendendo sobre sua história, arquitetura e cultura.
> * **Tarde:** Aproveite o sol e a areia na Praia da Moreninha pela última vez.
> * **Noite:** Desfrute de um jantar de despedida em um dos restaurantes da orla, saboreando a culinária local e as vistas deslumbrantes da Baía de Guanabara.

------------------------------------------------------------
